Land cover is the bio-physical material present on the Earth's surface. For example - water bodies, forests, bare soil, man-made surfaces.

Land use is the purpose or activity of the surface for human consumption. For example - agriculture, residential, commercial, conservation.

Land cover can be analysed for desired geographical region through satelite imagery. The trend of changed land cover can be used to understand the pattern of land use, and plan accordingly.

While a standard photo uses only Red, Green, and Blue channels, multi-spectral satellites like (Sentinel-2)[https://dataspace.copernicus.eu/data-collections/copernicus-sentinel-data/sentinel-2] capture "invisible" light across 12 distinct spectral bands. These extra bands, such as Infrared, allow us to detect "spectral signatures" that differentiate materials - like grass versus green paint.

For machine learning, we reshape this data into a matrix where each of the  H×W  pixels is a sample and each band is a feature. This high-dimensional depth enables algorithms like K-Means to group land covers with far greater precision than standard 3-band imagery.

In [ ]:
import rasterio
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt




PROJECT_ROOT = Path.cwd().parent
RAW_DATA_DIR = PROJECT_ROOT / 'data' / 'raw'
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

tiff_path = RAW_DATA_DIR / 'merged.tiff' # satellite very big file

# best practice: use a context manager so the file is always closed
with rasterio.open(tiff_path) as src:
    data = src.read()

# now data is 12 god arrays = 12 photographs and each photograph/array is a h*w matrix
# its a 3d numpy array/tensor
# so assuming 12 photos stacked left to right as red color = photo0, blue = photo1 and so on
# with each photo being its own 2d matrix  beffore reshape
# after reshape its 12 columsn of bands and each row is one sample = one pixel, shape (h*w,12)
# we arrange this way since ML convention is X (rows) = things you want to study (pixel/loc) and y (columns) = attributes of those things

print(f'\nRead data as matrix of shape {data.shape}')

data
# means we have say a point at h,w like x,y which has 12 values (one for each color)


Read data as matrix of shape (12, 1987, 2500)


array([[[0.    , 0.    , 0.    , ..., 0.0277, 0.0298, 0.0302],
        [0.    , 0.    , 0.    , ..., 0.0337, 0.0293, 0.0265],
        [0.    , 0.    , 0.    , ..., 0.0248, 0.0291, 0.0235],
        ...,
        [0.1072, 0.0764, 0.0362, ..., 0.0152, 0.0286, 0.0501],
        [0.1066, 0.0878, 0.022 , ..., 0.0164, 0.023 , 0.0594],
        [0.1162, 0.0892, 0.0608, ..., 0.0381, 0.0504, 0.0943]],

       [[0.    , 0.    , 0.    , ..., 0.0409, 0.0399, 0.0448],
        [0.    , 0.    , 0.    , ..., 0.0509, 0.0443, 0.0443],
        [0.    , 0.    , 0.    , ..., 0.0443, 0.0517, 0.028 ],
        ...,
        [0.1316, 0.1014, 0.0516, ..., 0.0337, 0.0553, 0.0818],
        [0.1282, 0.1054, 0.046 , ..., 0.0325, 0.0492, 0.084 ],
        [0.1498, 0.1112, 0.082 , ..., 0.0687, 0.071 , 0.1136]],

       [[0.    , 0.    , 0.    , ..., 0.0322, 0.0349, 0.0373],
        [0.    , 0.    , 0.    , ..., 0.0379, 0.0331, 0.032 ],
        [0.    , 0.    , 0.    , ..., 0.0331, 0.0354, 0.0245],
        ...,
        [0.1

In [11]:
# TODO: update these values from the shape of the matrix

N, H, W = data.shape # simple easy, aka tuple unpacking

# reshape the data to (H x W, N)
# the data in a row should correspond to a pixel

# sanity check
#random_pixel = (118, 256) # (h, w) Output: Array of 12 numbers (one per band)
#display(data[:, random_pixel[0], random_pixel[1]])

# init it was N H W now we want H W N, simply like index of shape tuple
# without transpose wed get 12 pixels from Band 0, NOT 12 bands for Pixel 0
data = data.transpose(1, 2, 0).reshape(H * W, N)
#display(data[random_pixel[0] * W + random_pixel[1]])
data.shape

(4967500, 12)

visually

**og (bands, h,w)**
`Memory: [B0P0, B0P1, ..., B0P_last, B1P0, B1P1, ..., B11P_last]`
**after transpose (h,w, bands)**
`Memory: [P0B0, P0B1, ..., P0B11, P1B0, P1B1, ..., P_lastB11]`
**after reshape (h*w, bands)**
```
Row 0: [P0B0, P0B1, ..., P0B11]
Row 1: [P1B0, P1B1, ..., P1B11]
```



In [ ]:
# TODO: import Pipeline, KMeans and StandardScaler from sklearn
# it is important to "normalize" the data before training (via std scaler)
# TODO: create a pipeline and use the "fit" method to train the model

k = 6  # choose number of land-cover clusters
pipeline = Pipeline([
    ("scaler", StandardScaler()),("kmeans", KMeans(n_clusters=k, random_state=42, n_init="auto"))
]).fit(data)
